In [1]:
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
import os

def calculate_pixel_snr(pixel_value, dark_current, exposure_time):
    """Calculate SNR for a single pixel."""
    signal = pixel_value - np.mean(dark_current)
    noise = np.std(dark_current)
    return max(signal, 0) / noise

def weighting_function(snr):
    """Define a weighting function based on SNR."""
    return snr / (snr + 1)  # This is a simple example; you can modify based on literature

def create_hdr_radiance_map(fluorescence_data, exposure_times, Sd, b):
    """Create HDR radiance map from multiple exposures."""
    num_exposures = len(exposure_times)
    shape = fluorescence_data[0].shape
    hdr_image = np.zeros(shape, dtype=float)
    weight_sum = np.zeros(shape, dtype=float)

    for i, exposure_time in enumerate(exposure_times):
        dark_current = Sd * exposure_time + b
        
        # Calculate SNR for each pixel
        snr = np.zeros(shape)
        for x in range(shape[0]):
            for y in range(shape[1]):
                snr[x, y] = calculate_pixel_snr(fluorescence_data[i][x, y], dark_current[x, y], exposure_time)
        
        # Calculate weights
        weights = weighting_function(snr)
        
        # Add weighted radiance to HDR image
        hdr_image += weights * (fluorescence_data[i] - dark_current) / exposure_time
        weight_sum += weights

    # Normalize
    hdr_image /= np.maximum(weight_sum, 1e-6)

    return hdr_image

def generate_hdr_report(hdr_image, output_directory, experiment_title):
    """Generate a PDF report for the HDR radiance map."""
    filename = os.path.join(output_directory, f"{experiment_title}_hdr_report.pdf")
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height - 50, f"HDR Radiance Map Report: {experiment_title}")

    # HDR image
    plt.figure(figsize=(8, 6))
    plt.imshow(np.log(hdr_image + 1), cmap='viridis')
    plt.colorbar(label='Log Radiance')
    plt.title('HDR Radiance Map (Log Scale)')
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
    buf.seek(0)
    
    img = ImageReader(buf)
    c.drawImage(img, 50, height - 400, width=400, height=300)

    # Statistics
    c.setFont("Helvetica", 12)
    c.drawString(50, height - 450, f"Min Radiance: {np.min(hdr_image):.2e}")
    c.drawString(50, height - 470, f"Max Radiance: {np.max(hdr_image):.2e}")
    c.drawString(50, height - 490, f"Mean Radiance: {np.mean(hdr_image):.2e}")
    c.drawString(50, height - 510, f"Dynamic Range: {np.max(hdr_image) / np.min(hdr_image):.2f}")

    c.save()
    print(f"HDR report saved as {filename}")

# Usage example
def process_hdr(data_directory, experiment_title):
    # Load data
    fluorescence_data, fl_exposure_times, Sd, b = load_data(data_directory)

    for condition, data in fluorescence_data.items():
        hdr_image = create_hdr_radiance_map(data, fl_exposure_times[condition], Sd, b)
        generate_hdr_report(hdr_image, data_directory, f"{experiment_title}_{condition}")

# Run the process
experiment_title = 'Water_immersed'
data_directory = f'data/{experiment_title}'
process_hdr(data_directory, experiment_title)

NameError: name 'load_data' is not defined